## Import package

In [35]:
import os
import openai

## 1) Assistant 생성

In [19]:


# OPENAI API 키를 환경 변수에 저장합니다.
# 주의: 실제 코드에서는 API 키를 코드에 직접 작성하는 것은 보안에 좋지 않습니다.
# 이 코드는 예시를 위한 것이며, 실제 사용시에는 키를 안전하게 관리해야 합니다.
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# API 키를 제공하여 OpenAI 클라이언트를 생성합니다.
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [20]:
# 어시스턴트 생성

assistant = client.beta.assistants.create(
    name="Recipe 전문가",
    instructions="사용자가 요리법을 요청할 때, 관련 레시피 데이터베이스에서 정보를 검색하고, 이해하기 쉽고 실행 가능한 조리법을 단계별로 설명 해주세요.",
    model="gpt-4-1106-preview",
    tools=[{"type":"retrieval"}]
)

## 2) Document Load

In [21]:
# upload the file
file = client.files.create(
    file=open(
        "asset/recipe.pdf",
        "rb",
    ),
    purpose="assistants",
)

# update Assistant
assistant = client.beta.assistants.update(
    assistant.id,
    file_ids=[file.id],
)

## 3) Thread 생성 및 Message 저장

In [22]:
# creating a thread
cooking_thread = client.beta.threads.create()

In [28]:
# 스레드에 첫 번째 메시지 추가
message = client.beta.threads.messages.create(
    thread_id=cooking_thread.id,
    role="user",
    content="칸쇼 새우 만드는 법 알려줘요"
)

## 4) 어시스턴트 실행(Run)

In [29]:
# Creating an Run

run = client.beta.threads.runs.create(
    assistant_id=assistant.id,
    thread_id=cooking_thread.id    
)

## 5) Message 출력

In [30]:
import time

# creating a function to check whether the Run is completed or not
def poll_run(run, thread):
    while run.status != "completed":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run
    
# waiting for the run to be completed
run = poll_run(run, cooking_thread)

In [31]:
# extracting the message
messages = client.beta.threads.messages.list(thread_id=cooking_thread.id)

for m in messages:
    print(f"{m.role}: {m.content[0].text.value}")

assistant: I've attempted to find the recipe for "칸쇼 새우" but encountered difficulties accessing the specific instructions you're requesting. It seems that the portion of the document I can view does not contain the detailed recipe steps, and my current tool capabilities do not allow me to search through the rest of the document effectively. 

If there's any other way I can assist you, please let me know! If you have a different recipe you'd like to inquire about, or if there's any other information related to "칸쇼 새우" or otherwise that you're looking for, feel free to ask.
user: 칸쇼 새우 만드는 법 알려줘요
assistant: 칸쇼 새우 조리 시 사용하는 야채 재료는 다음과 같습니다:

- 새우 200g
- 완두콩 2큰술
- 당근 ¼개

이 밖에도 소스 만들기에 사용하는 재료들은 다음과 같습니다:

- 고추기름 2큰술
- 대파 ¼대
- 마늘 ½큰술
- 생강 1작은술
- 설탕 2큰술
- 식초 1큰술【21†source】.
user: 야채 재료만 알려주세요
assistant: I apologize for the inconvenience, but it seems that I'm still not able to locate the specific recipe for "칸쇼 새우". The document appears to include information on the use and maintenance of an

In [32]:
# 두 번째 메시지 생성
message2 = client.beta.threads.messages.create(
    thread_id=cooking_thread.id,
    role="user",
    content="야채 재료만 알려주세요"
)

# Run 생성
run = client.beta.threads.runs.create(
    thread_id=cooking_thread.id,
    assistant_id=assistant.id
)

# Run 메서드가 완료될 때까지 대기
run = poll_run(run, cooking_thread)

# 메시지 추출
messages = client.beta.threads.messages.list(thread_id=cooking_thread.id)

for m in messages:
    print(f"{m.role}: {m.content[0].text.value}")


assistant: 앞서 언급했듯이, 칸쇼 새우를 만들 때 필요한 야채 재료는 다음과 같습니다:

- 완두콩 2큰술
- 당근 ¼개

그리고 소스를 만들 때 사용하는 재료들은 다음을 포함합니다:

- 고추기름 2큰술
- 대파 ¼대
- 마늘 ½큰술
- 생강 1작은술【20†source】.
user: 야채 재료만 알려주세요
assistant: I've attempted to find the recipe for "칸쇼 새우" but encountered difficulties accessing the specific instructions you're requesting. It seems that the portion of the document I can view does not contain the detailed recipe steps, and my current tool capabilities do not allow me to search through the rest of the document effectively. 

If there's any other way I can assist you, please let me know! If you have a different recipe you'd like to inquire about, or if there's any other information related to "칸쇼 새우" or otherwise that you're looking for, feel free to ask.
user: 칸쇼 새우 만드는 법 알려줘요
assistant: 칸쇼 새우 조리 시 사용하는 야채 재료는 다음과 같습니다:

- 새우 200g
- 완두콩 2큰술
- 당근 ¼개

이 밖에도 소스 만들기에 사용하는 재료들은 다음과 같습니다:

- 고추기름 2큰술
- 대파 ¼대
- 마늘 ½큰술
- 생강 1작은술
- 설탕 2큰술
- 식초 1큰술【21†source】.
user: 야채 재료만 알려주세요
assistant: I apologize f

In [33]:
# 두 번째 메시지 생성
message3 = client.beta.threads.messages.create(
    thread_id=cooking_thread.id,
    role="user",
    content="내 이름은 김윤웅이야"
)

# Run 생성
run = client.beta.threads.runs.create(
    thread_id=cooking_thread.id,
    assistant_id=assistant.id
)

# Run 메서드가 완료될 때까지 대기
run = poll_run(run, cooking_thread)

# 메시지 추출
messages = client.beta.threads.messages.list(thread_id=cooking_thread.id)

for m in messages:
    print(f"{m.role}: {m.content[0].text.value}")

assistant: 안녕하세요 김윤웅 님, 만나서 반가워요! 칸쇼 새우 조리법에 대해 도움이 필요하시면 언제든지 알려주세요. 다른 질문이나 요청 사항이 있으시면 말씀해주시길 바랍니다.
user: 내 이름은 김윤웅이야
assistant: 앞서 언급했듯이, 칸쇼 새우를 만들 때 필요한 야채 재료는 다음과 같습니다:

- 완두콩 2큰술
- 당근 ¼개

그리고 소스를 만들 때 사용하는 재료들은 다음을 포함합니다:

- 고추기름 2큰술
- 대파 ¼대
- 마늘 ½큰술
- 생강 1작은술【20†source】.
user: 야채 재료만 알려주세요
assistant: I've attempted to find the recipe for "칸쇼 새우" but encountered difficulties accessing the specific instructions you're requesting. It seems that the portion of the document I can view does not contain the detailed recipe steps, and my current tool capabilities do not allow me to search through the rest of the document effectively. 

If there's any other way I can assist you, please let me know! If you have a different recipe you'd like to inquire about, or if there's any other information related to "칸쇼 새우" or otherwise that you're looking for, feel free to ask.
user: 칸쇼 새우 만드는 법 알려줘요
assistant: 칸쇼 새우 조리 시 사용하는 야채 재료는 다음과 같습니다:

- 새우 200g
- 완두콩 2큰술
- 당근 ¼개

이 밖에도 소스 만들기에 사용하는 재료들은 다

In [34]:
# 두 번째 메시지 생성
message4 = client.beta.threads.messages.create(
    thread_id=cooking_thread.id,
    role="user",
    content="내 이름이 뭔지 알아?"
)

# Run 생성
run = client.beta.threads.runs.create(
    thread_id=cooking_thread.id,
    assistant_id=assistant.id
)

# Run 메서드가 완료될 때까지 대기
run = poll_run(run, cooking_thread)

# 메시지 추출
messages = client.beta.threads.messages.list(thread_id=cooking_thread.id)

for m in messages:
    print(f"{m.role}: {m.content[0].text.value}")

assistant: 네, 앞서 말씀하셨듯이 김윤웅 님이라고 하셨습니다. 무엇을 도와드릴까요?
user: 내 이름이 뭔지 알아?
assistant: 안녕하세요 김윤웅 님, 만나서 반가워요! 칸쇼 새우 조리법에 대해 도움이 필요하시면 언제든지 알려주세요. 다른 질문이나 요청 사항이 있으시면 말씀해주시길 바랍니다.
user: 내 이름은 김윤웅이야
assistant: 앞서 언급했듯이, 칸쇼 새우를 만들 때 필요한 야채 재료는 다음과 같습니다:

- 완두콩 2큰술
- 당근 ¼개

그리고 소스를 만들 때 사용하는 재료들은 다음을 포함합니다:

- 고추기름 2큰술
- 대파 ¼대
- 마늘 ½큰술
- 생강 1작은술【20†source】.
user: 야채 재료만 알려주세요
assistant: I've attempted to find the recipe for "칸쇼 새우" but encountered difficulties accessing the specific instructions you're requesting. It seems that the portion of the document I can view does not contain the detailed recipe steps, and my current tool capabilities do not allow me to search through the rest of the document effectively. 

If there's any other way I can assist you, please let me know! If you have a different recipe you'd like to inquire about, or if there's any other information related to "칸쇼 새우" or otherwise that you're looking for, feel free to ask.
user: 칸쇼 새우 만드는 법 알려줘요
assistant: 칸쇼 새우 조리 시 사용하는 야